# Organizer Quickstart
In the following cells you will find usage examples needed for managing the Tournament contract.

# Set Up (Always run this first)

In [ ]:
# Set to `True` if only querying the blockchain for information.
READ_ONLY = False

from src.rci_competition_utilities.organizer import *

# Enter Infura Project ID here.
INFURA_ID = '906ca3a563ce48fdb188ec4ebec1cf4f'

# Your account address.
PUBLIC_ADDRESS = ''
''

# Instantiate Organizer session. You will be prompted for your private key if `READ_ONLY` is set to False.
my_account = Organizer(read_only=READ_ONLY, network_url = network.polygon_mainnet, infura_id = INFURA_ID, account_address=PUBLIC_ADDRESS)
my_account.set_gas_price_in_gwei(2)

#POLYGON contracts
# Instantiate existing contracts
# REGISTRY
REGISTRY_CONTRACT_ADDRESS = '0x2a91f354F25f9F4C8AFe078c94A997bCd50DE7Ba'
my_account.registry_init(REGISTRY_CONTRACT_ADDRESS)

# VAULT
VAULT_CONTRACT_ADDRESS = '0xba163Dd4Cc83B3244099C5A300481F7f8Bc528F8'
my_account.vault_init(VAULT_CONTRACT_ADDRESS)

# TOKEN
TOKEN_CONTRACT_ADDRESS = '0x38036bbF1e95957aB04CFd47708c5ca293916FFe'
my_account.token_init(TOKEN_CONTRACT_ADDRESS)

# COMPETITON
COMPETITON_NAME = 'RCI_COMP_10'
COMPETITON_CONTRACT_ADDRESS = '0x4892A0cf39ff2F496C068F6130A5B410867e1Caf'
my_account.competition_init(COMPETITON_CONTRACT_ADDRESS)


In [ ]:
# Query and set gas price according to market prices
def gas_price_setter():
    market_gas_price_in_gwei = my_account.conversion_to_float(my_account.get_current_gas_price_in_wei()) * 1e9
    # print('Market gas price: {:.2f} Gwei'.format(market_gas_price_in_gwei))

    # Set slightly higher for faster transactions
    market_gas_price_in_gwei *= 1.5
    # print('Setting my gas price to: {:.2f} Gwei'.format(market_gas_price_in_gwei))

    my_account.set_gas_price_in_gwei(market_gas_price_in_gwei)

gas_price_setter();

In [ ]:
my_account.token.grantPermission(my_account.competition.address()); # only needs to be run once.

# CHALLENGE CYCLE BEGIN
## (Repeat for each challenge)

# PHASE 0

## Send tokens to sponsor Tournament contests (manually bypass Vault)

In [ ]:
sponsor_amount = 30
gas_price_setter()
my_account.sponsor(sponsor_amount);

# Dataset Creation

## Generate private-public key pair

In [ ]:
# Give a name for this key pair
challenge_number = my_account.get_latest_challenge_number()

asymmetric_key_gen("organizer_{}".format(challenge_number+1))
public_key_file_path = 'organizer_{}_public_key.pem'.format(challenge_number+1)

# PHASE 1

## Upload dataset and key and open for staking

In [ ]:
# Define dataset and public key file paths
# Create mock csv files and assign to 'train_file' and 'test_file'.
challenge_number = my_account.get_latest_challenge_number()
public_key_file_path = 'organizer_{}_public_key.pem'.format(challenge_number+1)

train_file = my_account.create_dataset()
time.sleep(2)
test_file = my_account.create_dataset()

next_challenge = my_account.get_latest_challenge_number() + 1

dataset_cid, key_cid = my_account.upload_datasets(train_file, test_file, public_key_file_path, 'challenge{}_dataset_{}'.format(next_challenge, datetime.now().strftime('%Y-%b-%d_%H:%M:%S')))

gas_price_setter()
my_account.competition.openChallenge(cid_to_hash(dataset_cid), cid_to_hash(key_cid))

# Set deadlines
from datetime import datetime, timedelta

current_milliseconds = int(datetime.now().timestamp() * 1000)
one_hour_in_milliseconds = timedelta(days=1, hours=0, minutes=0).seconds * 1000

deadline0 = current_milliseconds + one_hour_in_milliseconds
deadline1 = current_milliseconds + (2 * one_hour_in_milliseconds)
gas_price_setter()
my_account.competition.updateDeadlines(challenge_number, 0, deadline0)
my_account.competition.updateDeadlines(challenge_number, 1, deadline1);

In [ ]:
challenge_number = my_account.get_latest_challenge_number()
my_account.competition.getSubmitters(challenge_number)

# PHASE 2

In [ ]:
challenge_number = my_account.competition.getLatestChallengeNumber()

# Close submissions
gas_price_setter()
my_account.close_submission()

# Pay staking rewards
submitters = my_account.competition.getSubmitters(challenge_number)
staking_rewards_list = []
for s in submitters:
    staking_rewards_list.append(my_account.competition.computeStakingReward(s))

gas_price_setter()
my_account.competition.payStakingRewards(submitters, staking_rewards_list);



# PHASE 3

In [ ]:
gas_price_setter()
my_account.competition.advanceToPhase(3)

# Retrieve and decrypt submissions
# Select challenge number.
challenge_number = my_account.get_latest_challenge_number()

# Tell the function where to find the private key.
private_key_path = 'organizer_{}_private_key.pem'.format(challenge_number)

my_account.retrieve_submissions(challenge_number, private_key_path)

# A list of successful participants will be saved in: "successful_participants_phaseX.csv"
# Successful submissions will be saved in the folder: "decrypted_submissions_for_phaseX"
# A list of submissions with problems will be saved in: "problematic_submissions_for_phaseX.csv"

# Submit results
# Obtain a list of participants for this phase.
# participants = pd.read_csv("successful_participants_challenge{}.csv".format(challenge_number))['participant'].to_list()
participants = my_account.competition.getSubmitters(challenge_number)

# Create mock results to upload.
# This creates a results history for each participant, 
# uploads each individual results history, 
# then creates an overall csv file mapping each address to the IPFS cid of its results history.
results_cid = my_account.create_and_upload_results(challenge_number, participants)

# Record results cid on smart contract.
my_account.submit_results(results_cid)

# Create mock rewards csv.
# participants = pd.read_csv("successful_participants_challenge{}.csv".format(challenge_number))['participant'].to_list()
rewards_file_path = my_account.create_rewards_file(participants)

# Submit performance rewards payment to blockchain.
gas_price_setter()
my_account.pay_rewards(rewards_file_path)

# # Update Scores

submitters = my_account.competition.getSubmitters(challenge_number)
challenge_scores = []
tournament_scores = []


for s in submitters:
    challenge_scores.append(int(100 * random.random() * int(1e18)))
    tournament_scores.append(int(100 * random.random() * int(1e18)))


gas_price_setter()
my_account.competition.updateChallengeAndTournamentScores(submitters, challenge_scores, tournament_scores);



In [ ]:

for i in range(len(submitters)):
    s = submitters[i]
    cs = challenge_scores[i]
    ts = tournament_scores[i]

    cs_blockchain = my_account.competition.getChallengeScores(challenge_number, s)
    ts_blockchain = my_account.competition.getTournamentScores(challenge_number, s)

    print(s, cs, cs_blockchain, cs==cs_blockchain)
    print(s, ts, ts_blockchain, ts==ts_blockchain)
    



# PHASE 4 (Challenge Completed)

In [ ]:
gas_price_setter()
my_account.competition.advanceToPhase(4);

# END OF CHALLENGE CYCLE.
## Please return to CHALLENGE CYCLE BEGIN.

## New Structure For Holding Additional Values

A new structure for holding additional values for each participant per challenge has been added.
It is a mapping in the form (Item Number => Value).
The dictionary key is an Item Number instead of a variable or string so that we can more easily add Items later on.


In this implementation, I have allocated the Items as follows:
- Item Number 1 : Challenge Score
- Item Number 2 : Overall Competition Score
- Item Number 3 : Cumulative Rewards
- Item Number 4 : Rank


Examples:

If I want to get the Rank of participant 0xabcdef in challenge 5, I would call
`getInformation(5, 0xabcdef, 4)`

The total rewards received by a participant for each challenge is Participation + Performance rewards.


Floating-point values values need to go through the 18-decimals conversion.


These values can be recorded in the smart contract as follows.

Examples: 

I want to to store each participant's scores for challenge 22.
`updateInformationBatch(22, participants_list, 1, scores_list)`


I want to to store each participant's rank for challenge 22.
`updateInformationBatch(22, participants_list, 4, rank_list)`



## WRITE Additional Values

In [ ]:
# Generate mock information to store in the smart contract.

challenge_number = my_account.get_latest_challenge_number()

# participants = pd.read_csv("successful_participants_challenge{}.csv".format(challenge_number))['participant'].to_list()
participants = ['0x89390a8704C641a02119b9dc9CeE7243442774AB', '0xE1a3D0b0B38517c2C11f4B34Eb4a5cfa0cdd5E17']

rank_file = my_account.create_rank_file(participants)
my_account.upload_info(challenge_number, 3, rank_file, label='ranks')

reputation_file = my_account.create_data_file(participants, label='reputations')
my_account.upload_info(challenge_number, 4, reputation_file, label='reputations', from_wei = True)

scores_file = my_account.create_data_file(participants, label='scores')
my_account.upload_info(challenge_number, 5, scores_file, label='scores', from_wei = True)



## READ Additional Values

In [ ]:
challenge_number = my_account.get_latest_challenge_number()
print('Challenge number:', challenge_number)

# participants = pd.read_csv("successful_participants_challenge{}.csv".format(challenge_number))['participant'].to_list()
participants = ['0x89390a8704C641a02119b9dc9CeE7243442774AB', '0xE1a3D0b0B38517c2C11f4B34Eb4a5cfa0cdd5E17']
participant = participants[1]

print("Participant's current stake balance:", my_account.get_stake(participant))

staked_for_challenge = my_account.conversion_to_float(my_account.competition.getInformation(challenge_number, participant, 0))
print('Amount staked for this challenge:', staked_for_challenge)

participation_reward = my_account.conversion_to_float(my_account.competition.getInformation(challenge_number, participant, 1))
print('Participation Reward:', participation_reward)

performance_reward = my_account.conversion_to_float(my_account.competition.getInformation(challenge_number, participant, 2))
print('Performance Reward:', performance_reward)
                            
rank = my_account.competition.getInformation(challenge_number, participant, 3)
print('Rank:', rank)

reputation_score = my_account.conversion_to_float(my_account.competition.getInformation(challenge_number, participant, 4))
print('Reputation:', reputation_score)

score = my_account.conversion_to_float(my_account.competition.getInformation(challenge_number, participant, 5))
print('Score:', score)

## Setting gas price for transactions (optional)
When sending transactions, you may wish to specify the gas price you are willing to pay.
You can set this via the `my_account.set_gas_price_in_gwei` function.
For Polygon, 1 gwei (my_account.set_gas_price_in_gwei(1)) is usually enough. For Ropsten, 20 gwei (my_account.set_gas_price_in_gwei(20)) should be good enough when the network is not congested (transactions are not lagging).

You may look at websites such as https://ethgasstation.info for current gas price information.

Please note the various ETH denominations:
1 Ether = 1000000000 Gwei (9 zeroes) = 1000000000000000000 Wei (18 zeroes)

In [ ]:
# Set gas price to be used for all transactions.
my_account.set_gas_price_in_gwei(1)

## Authorize/Deauthorize another admin

In [ ]:
# Must be called  by role admin.
new_admin = '0xd36FE77D5E25022137A66Bea00fDBD922632FD7B'
main_admin_role_id = bytes(0)
gas_price_setter()
my_account.competition.grantRole(main_admin_role_id, new_admin);

# my_account.tournament.revokeRole(main_admin_role_id, new_admin)

### Deployment Code

In [ ]:

gas_price_setter()
# my_account.competition_deploy(10, 10, TOKEN_CONTRACT_ADDRESS, VAULT_CONTRACT_ADDRESS);

# print(my_account.registry.getCompetitionAddress('RCI_COMP_1'))
# print(my_account.registry.getTokenAddress())
# print(my_account.registry.getCompetitionVault(COMPETITON_CONTRACT_ADDRESS))
# print(my_account.registry.getCompetitionRulesLocation(COMPETITON_CONTRACT_ADDRESS).hex())

# print(my_account.registry.getCompetitionList())
my_account.registry.registerNewCompetition(COMPETITON_NAME, COMPETITON_CONTRACT_ADDRESS, VAULT_CONTRACT_ADDRESS, bytes(0));

### Test Score Updating

In [ ]:

# for i in range(len(submitters)):
#     s = submitters[i]
#     cs = challenge_scores[i]
#     ts = tournament_scores[i]

#     cs_blockchain = my_account.competition.getChallengeScores(challenge_number, s)
#     ts_blockchain = my_account.competition.getTournamentScores(challenge_number, s)

#     print(s, cs, cs_blockchain, cs==cs_blockchain)
#     print(s, ts, ts_blockchain, ts==ts_blockchain)
    

### Ethereum Ropsten Contract Addresses

In [ ]:
# #ETHEREUM contracts
# # Instantiate existing contracts
# # REGISTRY
# REGISTRY_CONTRACT_ADDRESS = '0x54ffe47B396d04FD567bC87788D085CF29Ac15E2'
# my_account.registry_init(REGISTRY_CONTRACT_ADDRESS)

# # VAULT
# VAULT_CONTRACT_ADDRESS = '0x6cf0Fd560fa99EaB59B8AB0e5186d792Bb1A7F6A'
# my_account.vault_init(VAULT_CONTRACT_ADDRESS)

# # TOKEN
# TOKEN_CONTRACT_ADDRESS = '0x6E4bb25176b5774969aD99181874d8F558676B6e'
# my_account.token_init(TOKEN_CONTRACT_ADDRESS)

# # COMPETITON
# TOURNAMENT_NAME = 'DAPP_TNMT_0'
# TOURNAMENT_CONTRACT_ADDRESS = '0xB2a9e06bDE7008B1dCDBEb7DF64bd570AaBd38EB'
# my_account.tournament_init(TOURNAMENT_CONTRACT_ADDRESS)

## POLYGON-MUMBAI Contracts
# # Instantiate existing contracts
# REGISTRY
# REGISTRY_CONTRACT_ADDRESS = '0x2a91f354F25f9F4C8AFe078c94A997bCd50DE7Ba'
# my_account.registry_init(REGISTRY_CONTRACT_ADDRESS)

# # VAULT
# VAULT_CONTRACT_ADDRESS = '0xba163Dd4Cc83B3244099C5A300481F7f8Bc528F8'
# my_account.vault_init(VAULT_CONTRACT_ADDRESS)

# # # TOKEN
# TOKEN_CONTRACT_ADDRESS = '0x81deF26864C97861038fC8F00a6977Bf715Db54D'
# my_account.token_init(TOKEN_CONTRACT_ADDRESS)

# # COMPETITON
# COMPETITION_NAME = 'DAPP_TNMT_7'
# COMPETITION_CONTRACT_ADDRESS = '0x28D6a3C14561c55CF7F15F92757bC7a50aCedc89'
# my_account.competition_init(COMPETITION_CONTRACT_ADDRESS)

# my_account.competition_deploy(10, 100, TOKEN_CONTRACT_ADDRESS, VAULT_CONTRACT_ADDRESS)